In [ ]:
import os
from rdflib import Graph, Namespace
import ipywidgets as widgets

shacl_output = widgets.Output()

# === SHACL-Regeln anwenden auf hochgeladene Datei ===
def apply_shacl_rules(instance_file_path: str) -> Graph:
    SH = Namespace("http://www.w3.org/ns/shacl#")
    SWEMLS = Namespace("https://w3id.org/semsys/ns/swemls#")
    shapes_dir = "Shapes"  # relative Pfadangabe im Repo

    shape_files = [
        "_generic-shapes.ttl", "A1-shapes.ttl", "A2-shapes.ttl", "A3-shapes.ttl",
        "F1-shapes.ttl", "F2-shapes.ttl", "F3-shapes.ttl", "F4-shapes.ttl",
        "I1-shapes.ttl", "I2-shapes.ttl", "I3-shapes.ttl", "I4-shapes.ttl",
        "I5-shapes.ttl", "I6-shapes.ttl", "I7-shapes.ttl", "O1-shapes.ttl",
        "O2-shapes.ttl", "O3-shapes.ttl", "O4-shapes.ttl", "T1-shapes.ttl",
        "T2-shapes.ttl", "T3-shapes.ttl", "T4-shapes.ttl", "T5-shapes.ttl",
        "T6-shapes.ttl", "T7-shapes.ttl", "T8-shapes.ttl", "T9-shapes.ttl",
        "T10-shapes.ttl", "T11-shapes.ttl", "T12-shapes.ttl", "T13-shapes.ttl",
        "T14-shapes.ttl", "T15-shapes.ttl", "T16-shapes.ttl", "T17-shapes.ttl",
        "T18-shapes.ttl", "T19-shapes.ttl", "T20-shapes.ttl", "T21-shapes.ttl",
        "T22-shapes.ttl", "T23-shapes.ttl", "Y1-shapes.ttl", "Y2-shapes.ttl", "Y4-shapes.ttl"
    ]

    g_instance = Graph()
    g_instance.parse(instance_file_path, format="turtle")

    triples_before = len(g_instance)

    for shape_file in shape_files:
        shape_path = os.path.join(shapes_dir, shape_file)
        g_shape = Graph()

        if not os.path.exists(shape_path):
            with shacl_output:
                print(f"⚠️ Shape file not found: {shape_path}")
            continue

        try:
            g_shape.parse(shape_path, format="turtle")
            with shacl_output:
                print(f"✅ Loaded SHACL shape file: {shape_file}")
        except Exception as e:
            with shacl_output:
                print(f"❌ Error loading {shape_file}: {e}")
            continue

        for rule in g_shape.subjects(predicate=SH.rule, object=None):
            for _, _, construct_query in g_shape.triples((rule, SH.construct, None)):
                query = str(construct_query)
                try:
                    g_instance.update(query)
                except Exception as e:
                    with shacl_output:
                        print(f"❌ Error executing rule from {shape_file}: {e}")

    triples_after = len(g_instance)
    with shacl_output:
        print("✅ All rules applied.")
        print(f"📊 Triples before: {triples_before}")
        print(f"📈 Triples after: {triples_after}")
        print(f"➕ Added: {triples_after - triples_before} triples")

    return g_instance


In [ ]:
import os
import ipywidgets as widgets
from IPython.display import display, clear_output
from rdflib import Graph

# === Globale Variablen ===
instance_file_path = None
output = widgets.Output()
result_graph = None

# === Auswahlfeld: Beispiel oder Upload ===
option_selector = widgets.ToggleButtons(
    options=[("Use example file", "example"), ("Upload your own", "upload")],
    description="Select input:",
    style={'description_width': 'initial'}
)

# === Upload-Widget (immer sichtbar) ===
upload_widget = widgets.FileUpload(
    accept='.ttl',
    multiple=False,
    description='Upload TTL file'
)

# === "Continue"-Button ===
continue_button = widgets.Button(description="Continue", button_style='primary')

# === Auswahlhandler ===
def on_option_change(change):
    output.clear_output()
    with output:
        if change['new'] == 'example':
            global instance_file_path
            instance_file_path = "Instance_Files/swemls-instances.ttl"
            print(f"📁 Example file selected:\n→ {instance_file_path}")
        elif change['new'] == 'upload':
            print("📤 Please upload a TTL file using the field below.")

option_selector.observe(on_option_change, names='value')

# === Upload-Handler ===
def on_upload(change):
    global instance_file_path
    if upload_widget.value:
        uploaded = upload_widget.value[0]
        file_name = uploaded['name']
        instance_file_path = file_name
        with open(file_name, 'wb') as f:
            f.write(uploaded['content'])
        output.clear_output()
        with output:
            print(f"✅ File uploaded and saved as:\n→ {file_name}")

upload_widget.observe(on_upload, names='value')

# === Continue-Button-Handler ===
def on_continue(b):
    global result_graph
    output.clear_output()
    if not instance_file_path:
        with output:
            print("⚠️ No file selected or uploaded.")
        return

    if not os.path.exists(instance_file_path):
        with output:
            print(f"❌ File not found on disk:\n→ {instance_file_path}")
        return

    try:
        if instance_file_path.endswith(".ttl") and not instance_file_path.startswith("Instance_Files/"):
            with output:
                print("🔍 SHACL rules will be applied to uploaded file.")
            result_graph = apply_shacl_rules(instance_file_path)
        else:
            g = Graph()
            g.parse(instance_file_path, format="turtle")
            result_graph = g

        with output:
            print(f"✅ RDF file successfully loaded!")
            print(f"📄 Triples in graph: {len(result_graph)}")
            print(f"🔗 Using file:\n→ {instance_file_path}")

    except Exception as e:
        with output:
            print(f"❌ Error parsing TTL file:\n→ {e}")

    # ⬇️ SHACL-Ausgabe sichtbar machen
    display(shacl_output)

continue_button.on_click(on_continue)

# === Anzeige aller Elemente ===
display(option_selector, upload_widget, continue_button, output)


In [ ]:
import ipywidgets as widgets
from rdflib.namespace import RDF
from IPython.display import display

# Outputs
query_interface_output = widgets.Output()
selection_output = widgets.Output()

# SPARQL Query-Feld
query_input = widgets.Textarea(
    value="""
PREFIX swemls: <https://w3id.org/semsys/ns/swemls#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT DISTINCT ?system
WHERE {
  ?system a swemls:System .
  ?system swemls:hasCorrespondingPattern ?pattern .
  FILTER(STRENDS(STR(?pattern), "O1"))
}
""",
    placeholder='Enter your SPARQL query here...',
    description='SPARQL Query:',
    layout=widgets.Layout(width='100%', height='150px'),
    style={'description_width': 'initial'}
)

# Buttons
run_query_button = widgets.Button(description="Run Query", button_style='primary')
confirm_button = widgets.Button(description="Confirm selection", button_style='success')

# Dropdown für Systeme
system_selector = widgets.Dropdown(
    options=[],
    description="Select system:",
    layout=widgets.Layout(width='50%')
)

# Globales Ergebnis
matched_systems = []

# Query-Handler
def on_query_run(b):
    global matched_systems
    query_interface_output.clear_output()
    selection_output.clear_output()
    system_selector.options = []

    if 'result_graph' not in globals() or result_graph is None:
        with query_interface_output:
            print("⚠️ RDF graph not loaded.")
        return

    try:
        results = result_graph.query(query_input.value)
    except Exception as e:
        with query_interface_output:
            print(f"❌ Query error: {e}")
        return

    matched_systems = []
    for row in results:
        uri = str(row.system)
        sys_id = uri.split("/")[-1]
        matched_systems.append((sys_id, uri))

    if not matched_systems:
        with query_interface_output:
            print("⚠️ No matching systems found.")
        return

    system_selector.options = [(sys_id, uri) for sys_id, uri in matched_systems]

    with query_interface_output:
        print(f"✅ Found {len(matched_systems)} matching system(s):")
        for i, (sys_id, _) in enumerate(matched_systems):
            print(f" {i+1}: {sys_id}")

# Auswahl-Handler
def on_confirm_selection(b):
    global selected_system_id, selected_system_uri
    selected_label = system_selector.label
    selected_uri = system_selector.value
    selected_system_id = selected_label
    selected_system_uri = selected_uri
    with selection_output:
        selection_output.clear_output()
        print(f"✅ You selected: {selected_system_id}")
        print(f"🔗 URI: {selected_system_uri}")

# Event-Bindings
run_query_button.on_click(on_query_run)
confirm_button.on_click(on_confirm_selection)

# Anzeigen
display(
    widgets.VBox([
        query_input,
        run_query_button,
        query_interface_output,
        widgets.HBox([system_selector, confirm_button]),
        selection_output
    ])
)


In [ ]:
import json
from rdflib import Graph, Namespace, URIRef, RDF
import ipywidgets as widgets
from IPython.display import display

json_output = widgets.Output()

def extract_and_export_selected_system():
    global selected_system_id, selected_system_uri, result_graph

    SWEMLS = Namespace("https://w3id.org/semsys/ns/swemls#")
    OPMW = Namespace("http://www.opmw.org/ontology#")
    RDFS = Namespace("http://www.w3.org/2000/01/rdf-schema#")

    rdf_type = URIRef("http://www.w3.org/1999/02/22-rdf-syntax-ns#type")
    ai_system_type = SWEMLS.System
    documentation_type = SWEMLS.Documentation
    paper_type = SWEMLS.Paper
    ml_component_type = SWEMLS.MachineLearningComponent
    kr_component_type = SWEMLS.KnowledgeRepresentationComponent
    data_type = SWEMLS.Data
    semantic_web_resource_type = SWEMLS.SemanticWebResource
    has_documentation = SWEMLS.hasDocumentation
    reports_on = SWEMLS.reports
    rdfs_label = RDFS.label

    g = result_graph

    # 1. System und zugehörige Paper finden
    ai_systems = [
        system for system in g.subjects(RDF.type, ai_system_type)
        if str(system).split("/")[-1] == selected_system_id
    ]

    system_to_paper = {}
    for paper in g.subjects(predicate=rdf_type, object=paper_type):
        for reported_system in g.objects(subject=paper, predicate=reports_on):
            system_id = str(reported_system).split("/")[-1]
            paper_id = str(paper).split("/")[-1]
            paper_metadata = {"id": paper_id, "metadata": {}}
            for pred, obj in g.predicate_objects(subject=paper):
                pred_name = pred.split("#")[-1] if "#" in pred else pred.split("/")[-1]
                paper_metadata["metadata"].setdefault(pred_name, []).append(str(obj))
            system_to_paper.setdefault(system_id, []).append(paper_metadata)

    # 2. System extrahieren
    for system in ai_systems:
        instance_data = {
            "id": str(system).split("/")[-1],
            "type": "System",
            "metadata": {},
            "relationships": {},
            "documentation": {},
            "papers": [],
            "steps": [],
            "variables": {}
        }

        for pred, obj in g.predicate_objects(subject=system):
            pred_name = pred.split("#")[-1] if "#" in pred else pred.split("/")[-1]
            if isinstance(obj, URIRef):
                instance_data["relationships"].setdefault(pred_name, []).append(str(obj).split("/")[-1])
            else:
                instance_data["metadata"][pred_name] = str(obj)

        # 3. Dokumentation extrahieren
        if "hasDocumentation" in instance_data["relationships"]:
            for doc_id in instance_data["relationships"]["hasDocumentation"]:
                doc_uri = URIRef(f"http://semantic-systems.net/swemls/{doc_id}")
                if (doc_uri, rdf_type, documentation_type) in g:
                    instance_data["documentation"]["id"] = doc_id
                    for doc_pred, doc_obj in g.predicate_objects(subject=doc_uri):
                        doc_pred_name = doc_pred.split("#")[-1] if "#" in doc_pred else doc_pred.split("/")[-1]
                        instance_data["documentation"][doc_pred_name] = str(doc_obj)

        # 4. Paper hinzufügen
        if selected_system_id in system_to_paper:
            instance_data["papers"] = system_to_paper[selected_system_id]

        # 5. Schritte (ML/KR) extrahieren
        for i in range(1, 11):
            for step_relation in [f"hasStepML{i}", f"hasStepKR{i}"]:
                if step_relation in instance_data["relationships"]:
                    for step_id in instance_data["relationships"][step_relation]:
                        step_uri = URIRef(f"http://semantic-systems.net/swemls/{step_id}")
                        step_type = "Unknown"
                        if (step_uri, rdf_type, ml_component_type) in g:
                            step_type = "Machine Learning"
                        elif (step_uri, rdf_type, kr_component_type) in g:
                            step_type = "Knowledge Representation"
                        step_data = {"id": step_id, "type": step_type, "metadata": {}}
                        for step_pred, step_obj in g.predicate_objects(subject=step_uri):
                            step_pred_name = step_pred.split("#")[-1] if "#" in step_pred else step_pred.split("/")[-1]
                            step_data["metadata"].setdefault(step_pred_name, []).append(str(step_obj))
                        instance_data["steps"].append(step_data)

        # 6. Variablen extrahieren
        for i in range(1, 11):
            for var_relation in [f"hasVariableData{i}", f"hasVariableSW{i}"]:
                if var_relation in instance_data["relationships"]:
                    for var_id in instance_data["relationships"][var_relation]:
                        var_uri = URIRef(f"http://semantic-systems.net/swemls/{var_id}")
                        label = None
                        for _, _, label_value in g.triples((var_uri, rdfs_label, None)):
                            label = str(label_value)
                            break
                        instance_data["variables"][var_relation] = {"id": var_id, "label": label}

        # 7. Speichern
        json_filename = f"{selected_system_id}.json"
        with open(json_filename, "w") as f:
            json.dump(instance_data, f, indent=4)

        with json_output:
            json_output.clear_output()
            print(f"✅ JSON successfully exported as: {json_filename}")
            print(json.dumps(instance_data, indent=2))  # Ausgabe für Überprüfung

# === Button zum Starten der Extraktion ===
extract_button = widgets.Button(description="Extract JSON", button_style="success")
extract_button.on_click(lambda b: extract_and_export_selected_system())

display(extract_button, json_output)


In [ ]:
import json
from IPython.display import display
import ipywidgets as widgets

pattern_output = widgets.Output()

# === Funktion zur ID-Deduplizierung ===
def deduplicate_ids(variables):
    seen_ids = {}
    updated_variables = {}

    for key, var in variables.items():
        original_id = var["id"]
        label = var.get("label", "")

        if original_id in seen_ids:
            seen_ids[original_id] += 1
            new_id = f"{original_id}_{seen_ids[original_id]}"
        else:
            seen_ids[original_id] = 1
            new_id = original_id

        updated_variables[key] = {
            "id": new_id,
            "label": label
        }

    return updated_variables

# === JSON-Datei einlesen und Variablen setzen ===
def load_instance_json(json_filename):
    global instance_data, pattern
    pattern = None

    try:
        with open(json_filename, "r") as json_file:
            extracted_data = json.load(json_file)
            instance_data = extracted_data

        # System-Label und Pattern-URI extrahieren
        system_label = instance_data["metadata"].get("label", "no label")
        raw_pattern_uri = instance_data["relationships"].get("hasCorrespondingPattern", None)

        if raw_pattern_uri:
            uri = raw_pattern_uri[0]
            pattern = uri.split(".")[-1] if "Pattern." in uri else uri.split("/")[-1]
            pattern_info = f"🧩 Pattern detected: {pattern}"
        else:
            pattern_info = "⚠️ No pattern detected"
            pattern = None

        # IDs deduplizieren
        instance_data["variables"] = deduplicate_ids(instance_data.get("variables", {}))

        with pattern_output:
            pattern_output.clear_output()
            print(f"✅ Loaded instance: {system_label}")
            print(pattern_info)

    except FileNotFoundError:
        with pattern_output:
            pattern_output.clear_output()
            print("❌ JSON file not found! Please extract an instance first.")
    except Exception as e:
        with pattern_output:
            pattern_output.clear_output()
            print(f"❌ Error loading JSON:\n→ {e}")

# === Button zur Ausführung ===
load_button = widgets.Button(description="Load Extracted JSON", button_style="primary")
load_button.on_click(lambda b: load_instance_json(f"{selected_system_id}.json"))

display(load_button, pattern_output)


In [ ]:
import os
import re
import json
from rdflib import Graph, Namespace, RDF
import ipywidgets as widgets
from IPython.display import display, clear_output

# === Globale Variablen ===
pattern_structure = {}
pattern_decision = None  # Wird gesetzt auf 'upload', 'template', oder 'auto'
pattern_extraction_output = widgets.Output()

# === Auswahlbuttons je nach Pattern-Zustand ===
pattern_option_output = widgets.Output()
pattern_upload_widget = widgets.FileUpload(
    accept=".ttl", multiple=False, description="Upload pattern TTL file"
)

pattern_confirm_button = widgets.Button(
    description="Extract Pattern", button_style="primary"
)

# === Pattern extrahieren ===
def extract_pattern_from_file(ttl_file_path):
    global pattern_structure

    SWEMLS = Namespace("https://w3id.org/semsys/ns/swemls#")
    OPMW = Namespace("http://www.opmw.org/ontology/")
    RES = Namespace("http://semantic-systems.net/swemls/")

    g = Graph()
    g.parse(ttl_file_path, format="turtle")

    pattern_structure = {"steps": {}, "variables": {}}

    def clean_uri(uri):
        label = uri.split("/")[-1].split("#")[-1]
        label = re.sub(r"Pattern\\.[A-Za-z0-9]+\\.", "", label)
        label = re.sub(r"^[A-Za-z0-9]+\\.", "", label)
        return label

    for ns in [SWEMLS, RES]:
        for step in g.subjects(RDF.type, ns.WorkflowTemplateProcessML):
            step_label = clean_uri(str(step))
            inputs = [clean_uri(str(v)) for v in g.objects(step, OPMW["uses"])]
            pattern_structure["steps"][step_label] = {"type": "ML", "uses": inputs, "outputs": []}

        for step in g.subjects(RDF.type, ns.WorkflowTemplateProcessKR):
            step_label = clean_uri(str(step))
            inputs = [clean_uri(str(v)) for v in g.objects(step, OPMW["uses"])]
            pattern_structure["steps"][step_label] = {"type": "KR", "uses": inputs, "outputs": []}

        for var_type in [ns.TemplateArtifactData, ns.TemplateArtifactSW]:
            for var in g.subjects(RDF.type, var_type):
                var_label = clean_uri(str(var))
                generated_by = [clean_uri(str(gen)) for gen in g.objects(var, OPMW["isGeneratedBy"])]

                for gen in generated_by:
                    if gen in pattern_structure["steps"]:
                        pattern_structure["steps"][gen]["outputs"].append(var_label)

                pattern_structure["variables"][var_label] = {"generated_by": generated_by} if generated_by else {}

# === Handler ===
def on_pattern_confirm(b):
    pattern_extraction_output.clear_output()
    with pattern_extraction_output:
        if pattern_decision == "upload":
            if pattern_upload_widget.value:
                uploaded = pattern_upload_widget.value[0]
                file_name = uploaded['name']
                with open(file_name, 'wb') as f:
                    f.write(uploaded['content'])
                extract_pattern_from_file(file_name)
                print("✅ Pattern successfully extracted from uploaded file:", file_name)
            else:
                print("⚠️ Please upload a pattern TTL file first.")
                return
        else:
            pattern_file = f"Patterns/{pattern}-pattern.ttl"
            if not os.path.exists(pattern_file):
                print(f"❌ Pattern file not found:", pattern_file)
                return
            extract_pattern_from_file(pattern_file)
            print("✅ Pattern successfully extracted from:", pattern_file)

        print("\n📦 Extracted Pattern Structure:")
        print(json.dumps(pattern_structure, indent=4))

pattern_confirm_button.on_click(on_pattern_confirm)

# === Anzeigeoptionen abhängig von Pattern ===
def show_pattern_options():
    pattern_option_output.clear_output()
    pattern_upload_widget.value.clear()
    global pattern_decision

    with pattern_option_output:
        if pattern == "no pattern":
            pattern_decision = "upload"
            print("⚠️ No corresponding pattern found.")
            print("📥 Please upload the pattern TTL file manually.")
            display(pattern_upload_widget, pattern_confirm_button)
        else:
            print(f"📌 Detected pattern: {pattern}")
            pattern_selector = widgets.ToggleButtons(
                options=[
                    ("Use predefined template", "template"),
                    ("Generate workflow automatically", "auto")
                ],
                description="Choose visualization mode:",
                style={'description_width': 'initial'}
            )

            def on_choice(change):
                global pattern_decision
                pattern_decision = change['new']
                print(f"\n🔧 Selected mode: {pattern_decision}")
                display(pattern_confirm_button)

            pattern_selector.observe(on_choice, names='value')
            display(pattern_selector)

# === Anzeigen ===
show_pattern_options()
display(pattern_option_output, pattern_extraction_output)